In [1]:
import pandas as pd
import numpy as np
import seaborn as sp
import matplotlib as mplot
from collections import defaultdict

In [2]:
weeksalesdb = pd.read_csv('train.csv')
stores = pd.read_csv('stores.csv')
features = pd.read_csv('features.csv')

In [3]:
features.drop(columns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], inplace = True)

In [4]:
#converts weeksalesdb rows into dict and merging departments
agg_sales = defaultdict(int)
for ind, sale in weeksalesdb.iterrows():
    agg_sales[str(sale['Store'])+'.'+sale['Date']] += sale['Weekly_Sales']

In [5]:
#converts back into db with updated sales value
storesales = pd.DataFrame(columns=['Store','Date','Weekly_Sales'])

for key, value in agg_sales.items():
    store, date = key.split('.')
    newrow = [store, date, value]
    storesales.loc[len(storesales)] = newrow

In [6]:
def convert_date(date):
    date = date.strip()
    date = date.replace('-','')
    date = date.strip()
    return int(date)
    
storesales['Date'] = storesales['Date'].apply(convert_date)

In [7]:
storesales.head(10)

,Store,Date,Weekly_Sales
0,1,20100205,1643690.90
1,1,20100212,1641957.44
2,1,20100219,1611968.17
3,1,20100226,1409727.59
4,1,20100305,1554806.68
5,1,20100312,1439541.59
6,1,20100319,1472515.79
7,1,20100326,1404429.92
8,1,20100402,1594968.28
9,1,20100409,1545418.53


In [9]:
storesales

Store      Date  Weekly_Sales
0        1  20100205    1643690.90
1        1  20100212    1641957.44
2        1  20100219    1611968.17
3        1  20100226    1409727.59
4        1  20100305    1554806.68
5        1  20100312    1439541.59
6        1  20100319    1472515.79
7        1  20100326    1404429.92
8        1  20100402    1594968.28
9        1  20100409    1545418.53
10       1  20100416    1466058.28
11       1  20100423    1391256.12
12       1  20100430    1425100.71
13       1  20100507    1603955.12
14       1  20100514    1494251.50
15       1  20100521    1399662.07
16       1  20100528    1432069.95
17       1  20100604    1615524.71
18       1  20100611    1542561.09
19       1  20100618    1503284.06
20       1  20100625    1422711.60
21       1  20100702    1492418.14
22       1  20100709    1546074.18
23       1  20100716    1448938.92
24       1  20100723    1385065.20
25       1  20100730    1371986.60
26       1  20100806    1605491.78
27       1  20100813    1508237.76
28       1  20100820    1513080.49
29       1  20100827    1449142.92
...    ...       ...           ...
6405    45  20120406     899479.43
6406    45  20120413     781970.60
6407    45  20120420     776661.74
6408    45  20120427     711571.88
6409    45  20120504     782300.68
6410    45  20120511     770487.37
6411    45  20120518     800842.28
6412    45  20120525     817741.17
6413    45  20120601     837144.63
6414    45  20120608     795133.00
6415    45  20120615     821498.18
6416    45  20120622     822569.16
6417    45  20120629     773367.71
6418    45  20120706     843361.10
6419    45  20120713     749817.08
6420    45  20120720     737613.65
6421    45  20120727     711671.58
6422    45  20120803     725729.51
6423    45  20120810     733037.32
6424    45  20120817     722496.93
6425    45  20120824     718232.26
6426    45  20120831     734297.87
6427    45  20120907     766512.66
6428    45  20120914     702238.27
6429    45  20120921     723086.20
6430    45  20120928     713173.95
6431    45  20121005     733455.07
6432    45  20121012     734464.36
6433    45  20121019     718125.53
6434    45  20121026     760281.43

[6435 rows x 3 columns]

In [9]:
max(storesales['Weekly_Sales']) - min(storesales['Weekly_Sales'])

3608700.1999999993